In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

from __future__ import print_function
import numpy as np

import keras
from keras.datasets import cifar10
from keras.layers import AveragePooling2D, Input, Flatten
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Model
import os

from sklearn.model_selection import train_test_split

Mixup Augmentation

In [0]:
class MixupAugmentation():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.4, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])
                
        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y

Cutout Augmentation

In [0]:
def random_cutout(p=0.5, v_l=0, v_h=255):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img
        
        w = 8
        mid_x = np.random.randint(0, img_w)
        mid_y = np.random.randint(0, img_h)
        
        c = np.random.uniform(v_l, v_h)
        input_img[mid_x - w : mid_x + w, mid_y - w : mid_y + w, :] = c

        return input_img

    return eraser

Standard Augmentation

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

def transform(new_img, shift, dirctn):
    assert dirctn in ['right', 'left', 'down', 'up'], 'dirctns should be top|up|left|right'
    img = new_img.copy()
    
    if dirctn == 'right':
        right_slice = img[:,:shift].shape
        img[:, shift:] = img[:, :-shift]
        img[:,:shift] = np.zeros(right_slice)
            
    if dirctn == 'left':
        left_slice = img[:, -shift:].shape
        img[:, :-shift] = img[:, shift:]
        img[:, -shift:] = np.zeros(left_slice)
    if dirctn == 'down':
        down_slice = img[:shift, :].shape
        img[shift:, :] = img[:-shift,:]
        img[:shift, :] = np.zeros(down_slice)
    if dirctn == 'up':
        upper_slice = img[-shift:,:].shape
        img[:-shift, :] = img[shift:, :]
        img[-shift:,:] = np.zeros(upper_slice)
    
    return img


def std_aug():
  def augment(image_in, p=0.5):
    K = 4
    k1 = np.random.randint(-K, K)
    k2 = np.random.randint(-K, K)
    
    #img_h, img_w, img_c = image_in.shape
    p_1 = np.random.rand()
    if k1!=0 and k2!=0:
      if k1>0:
        image_in = transform(image_in, shift=abs(k1), dirctn='up')
      else:
        image_in = transform(image_in, shift=abs(k1), dirctn='down')

      if k2>0:
        image_in = transform(image_in, shift=abs(k2), dirctn='right')
      else:
        image_in = transform(image_in, shift=abs(k2), dirctn='left')

    if p_1 > p:
      image_in = np.fliplr(image_in)
    return image_in
  return augment




RESNET20

In [0]:

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

 

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v2(input_shape, depth, num_classes=10):
    
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model



Data Preparation

In [0]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

input_shape = x_train.shape[1:]

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train, X_test, y_train, Y_test = train_test_split(x_train, y_train, stratify=y_train, test_size=0.8)

N = 10

y_train = keras.utils.to_categorical(y_train, N)
y_test = keras.utils.to_categorical(y_test, N)

Training Parameters

In [0]:
batch_size = 128 
epochs = 100
#N = 10

n=3
depth = n*6+2
model_type = 'ResNet%dv' % (depth)

Model

In [66]:
model = resnet_v2(input_shape=input_shape, depth=depth)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

#Standard Augmentation
datagen = ImageDataGenerator(
        preprocessing_function=std_aug())

datagen.fit(x_train)

#Cutout Augmentation
datagen = ImageDataGenerator(
        preprocessing_function=random_cutout(v_l=0, v_h=1))

datagen.fit(x_train)

#Mixup Augmentation
training_generator = MixupAugmentation(x_train, y_train, batch_size=batch_size, alpha=0.4, datagen = datagen)()

plot = model.fit_generator(generator=training_generator,
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1
                        )
             

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/100
78/78 [==============================] - 27s 350ms/step - loss: 2.3851 - accuracy: 0.3152 - val_loss: 2.8039 - val_accuracy: 0.2050
Epoch 2/100
78/78 [==============================] - 21s 272ms/step - loss: 2.1354 - accuracy: 0.4265 - val_loss: 4.2242 - val_accuracy: 0.1299
Epoch 3/100
78/78 [==============================] - 21s 272ms/step - loss: 2.0060 - accuracy: 0.4857 - val_loss: 2.3513 - val_accuracy: 0.3396
Epoch 4/100
78/78 [==============================] - 21s 273ms/step - loss: 1.9292 - accuracy: 0.5142 - val_loss: 2.8936 - val_accuracy: 0.2117
Epoch 5/100
78/78 [==============================] - 21s 272ms/step - loss: 1.8387 - accuracy: 0.5373 - val_loss: 2.0545 - val_accuracy: 0.4038
Epoch 6/100
78/78 [==============================] - 21s 273ms/step - loss: 1.7698 - accuracy: 0.5752 - val_loss: 1.7944 - val_accuracy: 0.4821
Epoch 7/100
78/78 [==============================] - 21s 272ms/step - loss: 1.6933 - accuracy: 0.6076 - val_loss: 1.7335 - val_accuracy:

Plot : Accuracy

In [0]:
import matplotlib.pyplot as plt

plt.figure(0)
plt.plot(plot.history['accuracy'], label='Training Accuracy')
plt.plot(plot.history['val_accuracy'], label='Test Accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

Plot : Loss

In [0]:
plt.plot(plot.history['loss'], label='Training Loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()